# **Processing downloaded files**

#### Product: Arctic Ocean Wave Hindcast
#### Dataset: cmems_mod_arc_wav_my_3km_PT1H-i
##### Check link: https://data.marine.copernicus.eu/product/ARCTIC_MULTIYEAR_WAV_002_013/files?subdataset=cmems_mod_arc_wav_my_3km_PT1H-i_202012

In [1]:
import copernicusmarine 
import xarray as xr
from pprint import pprint
import os
import time
import pandas as pd
import numpy as np
import pyproj
import shutil
from datetime import datetime, timedelta
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import folium
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Set the root of your project
os.chdir(r"C:\Users\LuisEnriqueBarrenoSu\OneDrive - CorPower Ocean AB\Desktop\Copernicus_Project")

In [50]:
class FileProcessor:
    def __init__(self, start_day, end_day):
        """
        Initialize the FileProcessor class.
        
        :param start_day: Begin of the dataset in str format (e.g., "2015-07-04").
        :param end_day: End of the dataset in str format (e.g., "2015-08-04").
        :param variables: List of variables to extract (e.g., ['VHM0', 'VTM10', 'VTM02'...]).
        """
        self.start_day = start_day
        self.end_day = end_day
        self.variables = None
        self.file_names = []
        self.data = None

    def load_files(self):
        """Load the downloaded files."""
        a1 = pd.DataFrame(pd.date_range(start = self.start_day, end = self.end_day, freq = "d"))
        series_dates = a1[0].apply(lambda x: str(x.date()).split("-"))
        file_names = list(series_dates.apply(lambda x: ("").join(x)).apply(lambda x: x+".nc"))
        self.file_names = file_names
        print(f"Files to concat: {file_names}")

    def concat_files(self):
        """Concatenate all files into a single Dataset."""
        if not self.file_names:
            raise ValueError("No files to process. Please run load_files() first.")
        list_size = len(self.file_names)
        ds_merge = xr.Dataset()
        for i in range(list_size):
            relative_path = os.path.join("data", "filtered_data", self.file_names[i])
            path = os.path.abspath(relative_path)
            ds = xr.open_dataset(path)
            ds_merge = xr.merge([ds_merge, ds])
        self.data = ds_merge
        self.variables = list(file_.data.data_vars)
        print("Files concatenated successfully.")


    def get_variables(self, var_):
        """
        Concatenate all files into a single Dataset.
        :param var_: variable name in str format (e.g., 'VHM0').
        """
        if self.data is None:
            raise ValueError("No data available. Please run concat_files() first.")

        var = self.data[var_]
        var = var.values
        
        lat = pd.DataFrame(self.data.lat.values)
        lon = pd.DataFrame(self.data.lon.values)
        
        a, b = lat.shape
        len_ = a * b
        coords_array = np.empty((len_), dtype=object)
        
        
        lat = lat.values.flatten()
        lon = lon.values.flatten()
        
        for i in range(len_):
            coords_array[i] = (lat[i], lon[i])
        df = pd.DataFrame()
        for row in range(a):
            for col in range(b):
                df = pd.concat([df, pd.DataFrame(var[:, row, col], index = pd.to_datetime(self.data.time.values))], axis = 1)
        
        df.columns = list(coords_array)
        return df

In [51]:
start_day = "2015-07-04"
end_day = "2015-07-06"
file_ = FileProcessor(start_day, end_day)

In [52]:
file_.load_files()

Files to concat: ['20150704.nc', '20150705.nc', '20150706.nc']


In [53]:
file_.concat_files()

Files concatenated successfully.


In [54]:
list(file_.data.data_vars)

['VHM0', 'VTM10']

In [56]:
VHM0 = file_.get_variables('VHM0')
VHM0

,"(59.00807532400741, -3.971224742186061)","(58.99162363678341, -3.934540047252207)","(58.97516117412991, -3.897896236738876)","(58.95868795965881, -3.86129327226354)","(58.94220401694944, -3.8247311153958443)","(58.925709369548585, -3.7882097276581375)","(58.90920404097047, -3.7517290705259887)","(58.89268805469672, -3.7152891054287807)","(58.87616143417641, -3.6788897937502085)","(58.85962420282604, -3.6425310968288174)",...,"(59.06382168388977, -3.3570145989215123)","(59.04716396481478, -3.3206057076233266)","(59.03049570125254, -3.2842379671425754)","(59.013816916763155, -3.2479113370079844)","(58.99712763487293, -3.211625776707494)","(58.98042787907466, -3.1753812456888597)","(58.963717672827386, -3.139177703360112)","(58.9469970395566, -3.10301510909013)","(58.93026600265408, -3.0668934222091604)","(58.913524585477994, -3.0308126020093415)"
2015-07-04 00:00:00,0.847345,0.831287,0.812555,0.792856,0.773660,0.833541,0.833541,0.822264,0.805758,0.777673,...,0.326650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.265410,0.271494
2015-07-04 01:00:00,0.898165,0.877058,0.855436,0.835039,0.815899,0.880657,0.880657,0.859359,0.838836,0.805342,...,0.334085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.294175,0.284706
2015-07-04 02:00:00,0.935550,0.909238,0.882232,0.855889,0.832920,0.913878,0.913878,0.900023,0.884529,0.862788,...,0.362107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.315630,0.285616
2015-07-04 03:00:00,1.014868,0.983702,0.950485,0.916565,0.888417,0.978393,0.978393,0.967711,0.958391,0.945256,...,0.394127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.318236,0.280528
2015-07-04 04:00:00,1.094135,1.064722,1.033185,0.998893,0.967166,1.043046,1.043046,1.023879,1.003466,0.977603,...,0.388815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.306767,0.265647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-07-06 19:00:00,0.762363,0.751423,0.743366,0.738294,0.734108,0.718517,0.718517,0.714064,0.694427,0.631006,...,0.450984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.026880,0.026397
2015-07-06 20:00:00,0.775554,0.765845,0.759252,0.755479,0.752143,0.738864,0.738864,0.732377,0.710190,0.646941,...,0.453379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027027,0.028719
2015-07-06 21:00:00,0.794472,0.784330,0.777008,0.771422,0.765920,0.746462,0.746462,0.735797,0.716323,0.664233,...,0.453911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027721,0.027888
2015-07-06 22:00:00,0.810464,0.799044,0.788582,0.778346,0.770993,0.757049,0.757049,0.751758,0.735190,0.681908,...,0.452869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027108,0.026544


In [57]:
VTM10 = file_.get_variables('VTM10')
VTM10

,"(59.00807532400741, -3.971224742186061)","(58.99162363678341, -3.934540047252207)","(58.97516117412991, -3.897896236738876)","(58.95868795965881, -3.86129327226354)","(58.94220401694944, -3.8247311153958443)","(58.925709369548585, -3.7882097276581375)","(58.90920404097047, -3.7517290705259887)","(58.89268805469672, -3.7152891054287807)","(58.87616143417641, -3.6788897937502085)","(58.85962420282604, -3.6425310968288174)",...,"(59.06382168388977, -3.3570145989215123)","(59.04716396481478, -3.3206057076233266)","(59.03049570125254, -3.2842379671425754)","(59.013816916763155, -3.2479113370079844)","(58.99712763487293, -3.211625776707494)","(58.98042787907466, -3.1753812456888597)","(58.963717672827386, -3.139177703360112)","(58.9469970395566, -3.10301510909013)","(58.93026600265408, -3.0668934222091604)","(58.913524585477994, -3.0308126020093415)"
2015-07-04 00:00:00,4.310108,4.290601,4.279974,4.270445,4.250831,4.181764,4.181764,4.142430,4.105296,4.069169,...,7.172072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.105946,1.974035
2015-07-04 01:00:00,4.253148,4.240921,4.233615,4.223000,4.199900,4.134041,4.134041,4.108446,4.070232,4.037596,...,6.983527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.248080,2.126037
2015-07-04 02:00:00,4.241649,4.233028,4.229027,4.223830,4.201395,4.120083,4.120083,4.077783,4.033319,3.983748,...,6.263766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.350419,2.219889
2015-07-04 03:00:00,4.160710,4.148750,4.144794,4.146430,4.133331,4.112647,4.112647,4.078619,4.038116,3.989059,...,5.646797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.379357,2.233837
2015-07-04 04:00:00,4.191433,4.165247,4.144801,4.132063,4.115315,4.153898,4.153898,4.132485,4.105417,4.069634,...,5.785903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.379148,2.223129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-07-06 19:00:00,5.899688,5.867376,5.843033,5.815169,5.790080,5.755880,5.755880,5.735854,5.704103,5.611786,...,5.808743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.727783,1.724537
2015-07-06 20:00:00,5.848927,5.810568,5.776873,5.740863,5.711907,5.666974,5.666974,5.671525,5.674658,5.607863,...,5.856024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.725259,1.707212
2015-07-06 21:00:00,5.738434,5.706279,5.681589,5.664404,5.656277,5.676146,5.676146,5.722121,5.700840,5.529602,...,5.910202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.715554,1.704662
2015-07-06 22:00:00,5.666016,5.642549,5.640834,5.658338,5.659136,5.620200,5.620200,5.604970,5.562309,5.437625,...,5.963769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.717822,1.716051
